In [1]:
import pandas as pd
from scipy.stats.stats import pearsonr
import statsmodels.api as sm
import numpy as np

# Table of Contents

Below analyses are for both individual and small group markets.

### Within Year Analysis
- [Mean, Variance, Min, Max, etc.](#a1)
- [Correlation Coefficients Within Year](#a2)
- [Costs 2016 ~ Premiums 2016 + Transfers 2016](#b6)
- [Costs 2015 ~ Premiums 2015 + Transfers 2015](#b7)
- [Costs 2014 ~ Premiums 2014 + Transfers 2014](#b8)

### Between Year Analysis
- [Transfers 2016 ~ Loss 2014 + Transfers 2015](#b1)
- [Transfers 2016 ~ Loss 2014 + Transfers 2014](#b2)
- [Transfers 2016 ~ Loss 2014 + Transfers 2014 + Loss 2015 + Transfers 2015](#b3)
- [Transfers 2015 ~ Loss 2014 + Transfers 2014](#b4)
- [Transfers 2016 ~ Loss 2015 + Transfers 2015](#b5)

### [Impact on Exiting the Market](#c1)



#  Cleaning the data

- Import the data

- Create the labels

- Filter the data to only include fields we want

- Drop companies with <2000 MM

- Calculate transfers, premiums, costs, and reinsurance (if individual group) per member month

In [2]:
data2014 = pd.read_csv("2014dataset.csv")
data2015 = pd.read_csv("2015dataset.csv")
data2016 = pd.read_csv("2016dataset.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,52,323,649,882,932,1092,1202) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,57,204,341,376,377,396,430,431,928,944,1006,1013,1151,1160,1169,1179,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,17,163,355,406,702,835,847,1083,1156,1183,1185,1200) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Create the labels.
# By index:
# label 2 is transfers
# label 3 is premiums
# label 4 is costs
# label 5 is member months
# label 6 is reinsurance (individual group only)

ind_labels_14 = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME",
                  "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
                  "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)"]
ind_labels_15 = [label.replace('15', '16') for label in ind_labels_14]
ind_labels_16 = [label.replace('15', '17') for label in ind_labels_14]

sg_labels_14 = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME",
                 "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)",
                  "1.1Directpremiumwritten7HealthInsuranceSMALLGROUPTotalasof3/31/15",
                  "2.16Totalincurredclaims7HealthInsuranceSMALLGROUPTotalasof3/31/15",
                  "7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"]
sg_labels_15 = [label.replace('15', '16') for label in sg_labels_14]
sg_labels_16 = [label.replace('15', '17') for label in sg_labels_14]

In [4]:
# Filter the data to include only the fields we want

ind_data_14 = data2014[ind_labels_14].dropna()
ind_data_15 = data2015[ind_labels_15].dropna()
ind_data_16 = data2016[ind_labels_16].dropna()
sg_data_14 = data2014[sg_labels_14].dropna()
sg_data_15 = data2015[sg_labels_15].dropna()
sg_data_16 = data2016[sg_labels_16].dropna()

In [5]:
# Drop companies with <2000 MM
ind_data_14 = ind_data_14.where(ind_data_14[ind_labels_14[5]] > 2000).dropna()
ind_data_15 = ind_data_15.where(ind_data_15[ind_labels_15[5]] > 2000).dropna()
ind_data_16 = ind_data_16.where(ind_data_16[ind_labels_16[5]] > 2000).dropna()
sg_data_14 = sg_data_14.where(sg_data_14[sg_labels_14[5]] > 2000).dropna()
sg_data_15 = sg_data_15.where(sg_data_15[sg_labels_15[5]] > 2000).dropna()
sg_data_16 = sg_data_16.where(sg_data_16[sg_labels_16[5]] > 2000).dropna()

In [6]:
# Calculate transfers, premiums, costs, and reinsurance (if individual group) per member month
def calculate_pmm(df, df_labels, group):
    df["TransfersPMM"] = df[df_labels[2]] / df[df_labels[5]]
    df["PremiumsPMM"] = abs(df[df_labels[3]] / df[df_labels[5]])
    df["CostsPMM"] = abs(df[df_labels[4]] / df[df_labels[5]])
    if group == "i":
        df["ReinsurancePMM"] = df[df_labels[6]] / df[df_labels[5]]
        df["LossPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"] + df["ReinsurancePMM"]
    elif group == "sg":
        df["LossPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"]
    return df

ind_data_14 = calculate_pmm(ind_data_14, ind_labels_14, "i")
ind_data_15 = calculate_pmm(ind_data_15, ind_labels_15, "i")
ind_data_16 = calculate_pmm(ind_data_16, ind_labels_16, "i")
sg_data_14 = calculate_pmm(sg_data_14, sg_labels_14, "sg")
sg_data_15 = calculate_pmm(sg_data_15, sg_labels_15, "sg")
sg_data_16 = calculate_pmm(sg_data_16, sg_labels_16, "sg")

In [7]:
# Combine the dataframes by left merging

ind_data = ind_data_14.merge(ind_data_15, on='HIOS ID', how='outer', suffixes=('_14', '_15'))\
.merge(ind_data_16, on='HIOS ID', how='outer')

In [8]:
# Do the same thing for small group

sg_data = sg_data_14.merge(sg_data_15, on='HIOS ID', how='outer', suffixes=('_14', '_15'))\
.merge(sg_data_16, on='HIOS ID', how='outer')

# Within-year analysis

- Mean, Variance, Min, Max

- Correlation coefficient analysis within each year

<a id='a1'></a>
### Mean, Variance, Min, Max

In [9]:
# Mean

print("Year&Group\tAverage Costs\tAverage Premiums\tAverage Absolute Transfers\n")
print("SG 2016\t%d\t%d\%d\n" % (np.mean(sg_data['CostsPMM']), np.mean(sg_data['PremiumsPMM']), np.mean(abs(sg_data['TransfersPMM']))))
print("Ind 2016\t%d\t%d\%d\n" % (np.mean(ind_data['CostsPMM']), np.mean(ind_data['PremiumsPMM']), np.mean(abs(ind_data['TransfersPMM']))))
print("SG 2015\t%d\t%d\%d\n" % (np.mean(sg_data['CostsPMM_15']), np.mean(sg_data['PremiumsPMM_15']), np.mean(abs(sg_data['TransfersPMM_15']))))
print("Ind 2015\t%d\t%d\%d\n" % (np.mean(ind_data['CostsPMM_15']), np.mean(ind_data['PremiumsPMM_15']), np.mean(abs(ind_data['TransfersPMM_15']))))
print("SG 2014\t%d\t%d\%d\n" % (np.mean(sg_data['CostsPMM_14']), np.mean(sg_data['PremiumsPMM_14']), np.mean(abs(sg_data['TransfersPMM_14']))))
print("Ind 2014\t%d\t%d\%d\n" % (np.mean(ind_data['CostsPMM_14']), np.mean(ind_data['PremiumsPMM_14']), np.mean(abs(ind_data['TransfersPMM_14']))))

# Variance
print("Year&Group\tVar Costs\tVar Premiums\tVar Absolute Transfers\n")
print("SG 2016\t%d\t%d\%d\n" % (np.var(sg_data['CostsPMM']), np.var(sg_data['PremiumsPMM']), np.var(abs(sg_data['TransfersPMM']))))
print("Ind 2016\t%d\t%d\%d\n" % (np.var(ind_data['CostsPMM']), np.var(ind_data['PremiumsPMM']), np.var(abs(ind_data['TransfersPMM']))))
print("SG 2015\t%d\t%d\%d\n" % (np.var(sg_data['CostsPMM_15']), np.var(sg_data['PremiumsPMM_15']), np.var(abs(sg_data['TransfersPMM_15']))))
print("Ind 2015\t%d\t%d\%d\n" % (np.var(ind_data['CostsPMM_15']), np.var(ind_data['PremiumsPMM_15']), np.var(abs(ind_data['TransfersPMM_15']))))
print("SG 2014\t%d\t%d\%d\n" % (np.var(sg_data['CostsPMM_14']), np.var(sg_data['PremiumsPMM_14']), np.var(abs(sg_data['TransfersPMM_14']))))
print("Ind 2014\t%d\t%d\%d\n" % (np.var(ind_data['CostsPMM_14']), np.var(ind_data['PremiumsPMM_14']), np.var(abs(ind_data['TransfersPMM_14']))))

# Min
print("Year&Group\tVar Costs\tVar Premiums\tVar Absolute Transfers\n")
print("SG 2016\t%d\t%d\%d\n" % (np.min(sg_data['CostsPMM']), np.min(sg_data['PremiumsPMM']), np.min(abs(sg_data['TransfersPMM']))))
print("Ind 2016\t%d\t%d\%d\n" % (np.min(ind_data['CostsPMM']), np.min(ind_data['PremiumsPMM']), np.min(abs(ind_data['TransfersPMM']))))
print("SG 2015\t%d\t%d\%d\n" % (np.min(sg_data['CostsPMM_15']), np.min(sg_data['PremiumsPMM_15']), np.min(abs(sg_data['TransfersPMM_15']))))
print("Ind 2015\t%d\t%d\%d\n" % (np.min(ind_data['CostsPMM_15']), np.min(ind_data['PremiumsPMM_15']), np.min(abs(ind_data['TransfersPMM_15']))))
print("SG 2014\t%d\t%d\%d\n" % (np.min(sg_data['CostsPMM_14']), np.min(sg_data['PremiumsPMM_14']), np.min(abs(sg_data['TransfersPMM_14']))))
print("Ind 2014\t%d\t%d\%d\n" % (np.min(ind_data['CostsPMM_14']), np.min(ind_data['PremiumsPMM_14']), np.min(abs(ind_data['TransfersPMM_14']))))

# Max
print("Year&Group\tVar Costs\tVar Premiums\tVar Absolute Transfers\n")
print("SG 2016\t%d\t%d\%d\n" % (np.max(sg_data['CostsPMM']), np.max(sg_data['PremiumsPMM']), np.max(abs(sg_data['TransfersPMM']))))
print("Ind 2016\t%d\t%d\%d\n" % (np.max(ind_data['CostsPMM']), np.max(ind_data['PremiumsPMM']), np.max(abs(ind_data['TransfersPMM']))))
print("SG 2015\t%d\t%d\%d\n" % (np.max(sg_data['CostsPMM_15']), np.max(sg_data['PremiumsPMM_15']), np.max(abs(sg_data['TransfersPMM_15']))))
print("Ind 2015\t%d\t%d\%d\n" % (np.max(ind_data['CostsPMM_15']), np.max(ind_data['PremiumsPMM_15']), np.max(abs(ind_data['TransfersPMM_15']))))
print("SG 2014\t%d\t%d\%d\n" % (np.max(sg_data['CostsPMM_14']), np.max(sg_data['PremiumsPMM_14']), np.max(abs(sg_data['TransfersPMM_14']))))
print("Ind 2014\t%d\t%d\%d\n" % (np.max(ind_data['CostsPMM_14']), np.max(ind_data['PremiumsPMM_14']), np.max(abs(ind_data['TransfersPMM_14']))))

Year&Group	Average Costs	Average Premiums	Average Absolute Transfers

SG 2016	339	422\28

Ind 2016	416	375\63

SG 2015	319	405\27

Ind 2015	391	345\51

SG 2014	304	386\14

Ind 2014	366	314\41

Year&Group	Var Costs	Var Premiums	Var Absolute Transfers

SG 2016	6986	4901\1560

Ind 2016	53043	6275\9792

SG 2015	6893	5304\1459

Ind 2015	35317	6052\4801

SG 2014	6036	6627\660

Ind 2014	33494	8420\3209

Year&Group	Var Costs	Var Premiums	Var Absolute Transfers

SG 2016	119	185\0

Ind 2016	78	119\0

SG 2015	73	102\0

Ind 2015	24	114\0

SG 2014	111	151\0

Ind 2014	86	46\0

Year&Group	Var Costs	Var Premiums	Var Absolute Transfers

SG 2016	964	711\369

Ind 2016	3401	826\1128

SG 2015	849	656\251

Ind 2015	2280	701\833

SG 2014	927	1162\190

Ind 2014	1634	747\436



<a id='a2'></a>
### Correlation Coefficients within a year

In [10]:
# Correlation coefficient analysis within each year

def within_year_analysis(df, year, group):
    print("%d | %s | N: %d" % (year, group, len(df)))
    r1 = pearsonr(df["TransfersPMM"], df["PremiumsPMM"])
    r2 = pearsonr(df["CostsPMM"], df["TransfersPMM"])
    r3 = pearsonr(df["CostsPMM"], df["PremiumsPMM"])
    r4 = pearsonr(df["LossPMM"], df["TransfersPMM"])
    print("Premiums v Transfers | r: %f, p: %f" % (r1[0], r1[1]))
    print("Costs v Transfers | r: %f, p: %f" % (r2[0], r2[1]))
    print("Costs v Premiums | r: %f, p: %f" % (r3[0], r3[1]))
    print("Loss v Transfers | r: %f, p: %f\n" % (r4[0], r4[1]))
    
within_year_analysis(ind_data_14, 2014, "individual")
within_year_analysis(ind_data_15, 2015, "individual")
within_year_analysis(ind_data_16, 2016, "individual")
within_year_analysis(sg_data_14, 2014, "small group")
within_year_analysis(sg_data_15, 2015, "small group")
within_year_analysis(sg_data_16, 2016, "small group")

2014 | individual | N: 403
Premiums v Transfers | r: 0.289779, p: 0.000000
Costs v Transfers | r: 0.666629, p: 0.000000
Costs v Premiums | r: 0.629953, p: 0.000000
Loss v Transfers | r: 0.071102, p: 0.154237

2015 | individual | N: 453
Premiums v Transfers | r: 0.399257, p: 0.000000
Costs v Transfers | r: 0.719635, p: 0.000000
Costs v Premiums | r: 0.631623, p: 0.000000
Loss v Transfers | r: 0.043465, p: 0.356017

2016 | individual | N: 420
Premiums v Transfers | r: 0.422051, p: 0.000000
Costs v Transfers | r: 0.831887, p: 0.000000
Costs v Premiums | r: 0.637192, p: 0.000000
Loss v Transfers | r: -0.221924, p: 0.000004

2014 | small group | N: 539
Premiums v Transfers | r: 0.009755, p: 0.821231
Costs v Transfers | r: 0.152365, p: 0.000385
Costs v Premiums | r: 0.766419, p: 0.000000
Loss v Transfers | r: 0.317454, p: 0.000000

2015 | small group | N: 542
Premiums v Transfers | r: 0.113839, p: 0.007983
Costs v Transfers | r: 0.194727, p: 0.000005
Costs v Premiums | r: 0.644318, p: 0.0000

# Between-year analysis

<a id='b1'></a>
### Transfers 2016 ~ Loss 2014 + Transfers 2014

In [11]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2014 + Transfers 2014

no_nulls = ind_data[['LossPMM_14', 'TransfersPMM_14', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     92.94
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           6.47e-32
Time:                        13:49:19   Log-Likelihood:                -1744.9
No. Observations:                 293   AIC:                             3496.
Df Residuals:                     290   BIC:                             3507.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               3.0379      6.070      0.501      0.617      -8.908      14.984
LossPMM_14         -0.2083      0.079     -2.644      0.009      -0.363      -0.053
TransfersPMM_14     1.0920      0.080     13.634      0.000       0.934       1.250
==============================================================================
Omnibus:                      184.317   Durbin-Watson:                   2.085
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3133.297
Skew:                           2.213   Prob(JB):                         0.00
Kurtosis:                      18.397   Cond. No.                         93.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# SMALL GROUP Transfers 2016 = Loss 2014 + Transfers 2014

no_nulls = sg_data[['LossPMM_14', 'TransfersPMM_14', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     81.88
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           7.56e-31
Time:                        13:49:19   Log-Likelihood:                -2157.7
No. Observations:                 432   AIC:                             4321.
Df Residuals:                     429   BIC:                             4334.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -4.3722      3.451     -1.267      0.206     -11.156       2.411
LossPMM_14          0.0003      0.036      0.010      0.992      -0.070       0.070
TransfersPMM_14     0.7471      0.062     11.986      0.000       0.625       0.870
==============================================================================
Omnibus:                      237.448   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4739.555
Skew:                          -1.899   Prob(JB):                         0.00
Kurtosis:                      18.776   Cond. No.                         189.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b2'></a>
### Transfers 2016 ~ Loss 2014 + Transfers 2015

In [13]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2014 + Transfers 2015

no_nulls = ind_data[['LossPMM_14', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     546.9
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.25e-98
Time:                        13:49:19   Log-Likelihood:                -1567.5
No. Observations:                 290   AIC:                             3141.
Df Residuals:                     287   BIC:                             3152.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.6661      3.515      0.758      0.449      -4.253       9.585
LossPMM_14         -0.1604      0.048     -3.375      0.001      -0.254      -0.067
TransfersPMM_15     1.1779      0.036     32.998      0.000       1.108       1.248
==============================================================================
Omnibus:                       68.662   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              774.635
Skew:                           0.555   Prob(JB):                    6.17e-169
Kurtosis:                      10.929   Cond. No.                         100.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
# SMALL GROUP Transfers 2016 = Loss 2014 + Transfers 2015

no_nulls = sg_data[['LossPMM_14', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     175.8
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.87e-56
Time:                        13:49:19   Log-Likelihood:                -2093.7
No. Observations:                 431   AIC:                             4193.
Df Residuals:                     428   BIC:                             4206.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -6.2944      2.876     -2.189      0.029     -11.947      -0.642
LossPMM_14          0.0290      0.030      0.971      0.332      -0.030       0.088
TransfersPMM_15     0.7650      0.042     18.032      0.000       0.682       0.848
==============================================================================
Omnibus:                      334.162   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14591.083
Skew:                          -2.831   Prob(JB):                         0.00
Kurtosis:                      30.936   Cond. No.                         181.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b3'></a>
### Transfers 2016 ~ Loss 2014 + Transfers 2014 + Loss 2015 + Transfers 2015

In [15]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2014 + Transfers 2014 + Loss 2015 + Transfers 2015

no_nulls = ind_data[['LossPMM_14', 'TransfersPMM_14', 'LossPMM_15',
                     'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_14', 'LossPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     275.3
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.46e-96
Time:                        13:49:19   Log-Likelihood:                -1565.9
No. Observations:                 290   AIC:                             3142.
Df Residuals:                     285   BIC:                             3160.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.0190      3.532      0.572      0.568      -4.933       8.971
LossPMM_14         -0.1135      0.056     -2.023      0.044      -0.224      -0.003
TransfersPMM_14    -0.0179      0.069     -0.259      0.796      -0.154       0.118
LossPMM_15         -0.0702      0.040     -1.762      0.079      -0.149       0.008
TransfersPMM_15     1.1892      0.053     22.485      0.000       1.085       1.293
==============================================================================
Omnibus:                       47.642   Durbin-Watson:                   2.058
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              383.582
Skew:                           0.300   Prob(JB):                     5.08e-84
Kurtosis:                       8.602   Cond. No.                         123.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# SMALL GROUP Transfers 2016 = Loss 2014 + Transfers 2014 + Loss 2015 + Transfers 2015

no_nulls = sg_data[['LossPMM_14', 'TransfersPMM_14', 'LossPMM_15',
                     'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_14', 'LossPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.471
Method:                 Least Squares   F-statistic:                     96.74
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.69e-58
Time:                        13:49:19   Log-Likelihood:                -2083.7
No. Observations:                 431   AIC:                             4177.
Df Residuals:                     426   BIC:                             4198.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -6.2978      3.270     -1.926      0.055     -12.725       0.130
LossPMM_14         -0.0542      0.037     -1.462      0.144      -0.127       0.019
TransfersPMM_14     0.2689      0.067      4.034      0.000       0.138       0.400
LossPMM_15          0.0874      0.036      2.418      0.016       0.016       0.158
TransfersPMM_15     0.5865      0.058     10.184      0.000       0.473       0.700
==============================================================================
Omnibus:                      304.846   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11036.635
Skew:                          -2.512   Prob(JB):                         0.00
Kurtosis:                      27.276   Cond. No.                         296.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b4'></a>
### Transfers 2015 ~ Loss 2014 + Transfers 2014

In [17]:
# INDIVIDUAL GROUP Transfers 2015 = Loss 2014 + Transfers 2014

no_nulls = ind_data[['LossPMM_14', 'TransfersPMM_14', 'TransfersPMM_15']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TransfersPMM_15   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     198.8
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.72e-59
Time:                        13:49:19   Log-Likelihood:                -2079.3
No. Observations:                 376   AIC:                             4165.
Df Residuals:                     373   BIC:                             4176.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -2.4764      3.351     -0.739      0.460      -9.066       4.114
LossPMM_14          0.0220      0.042      0.529      0.597      -0.060       0.104
TransfersPMM_14     0.9135      0.046     19.712      0.000       0.822       1.005
==============================================================================
Omnibus:                       86.041   Durbin-Watson:                   1.932
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1881.247
Skew:                           0.252   Prob(JB):                         0.00
Kurtosis:                      13.947   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# SMALL GROUP Transfers 2015 = Loss 2014 + Transfers 2014

no_nulls = sg_data[['LossPMM_14', 'TransfersPMM_14', 'TransfersPMM_15']].dropna()
X = no_nulls[['LossPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TransfersPMM_15   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     122.9
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           4.33e-44
Time:                        13:49:19   Log-Likelihood:                -2458.8
No. Observations:                 499   AIC:                             4924.
Df Residuals:                     496   BIC:                             4936.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -6.7551      2.847     -2.372      0.018     -12.350      -1.161
LossPMM_14          0.0476      0.029      1.617      0.107      -0.010       0.105
TransfersPMM_14     0.7656      0.054     14.096      0.000       0.659       0.872
==============================================================================
Omnibus:                       91.685   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1165.942
Skew:                          -0.323   Prob(JB):                    6.59e-254
Kurtosis:                      10.461   Cond. No.                         178.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b5'></a>
### Transfers 2016 ~ Loss 2015 + Transfers 2015

In [19]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2015 + Transfers 2015

no_nulls = ind_data[['LossPMM_15', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     530.8
Date:                Thu, 15 Nov 2018   Prob (F-statistic):          5.90e-108
Time:                        13:49:19   Log-Likelihood:                -1974.3
No. Observations:                 362   AIC:                             3955.
Df Residuals:                     359   BIC:                             3966.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.9639      3.006      0.321      0.749      -4.948       6.876
LossPMM_15         -0.1297      0.033     -3.931      0.000      -0.195      -0.065
TransfersPMM_15     1.1158      0.034     32.412      0.000       1.048       1.184
==============================================================================
Omnibus:                       54.291   Durbin-Watson:                   2.122
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              315.618
Skew:                           0.427   Prob(JB):                     2.91e-69
Kurtosis:                       7.494   Cond. No.                         91.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# SMALL GROUP Transfers 2016 = Loss 2015 + Transfers 2015

no_nulls = sg_data[['LossPMM_15', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['LossPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     156.5
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.95e-52
Time:                        13:49:19   Log-Likelihood:                -2256.4
No. Observations:                 458   AIC:                             4519.
Df Residuals:                     455   BIC:                             4531.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.9256      3.012     -2.963      0.003     -14.845      -3.006
LossPMM_15          0.0565      0.030      1.904      0.058      -0.002       0.115
TransfersPMM_15     0.6236      0.045     13.952      0.000       0.536       0.711
==============================================================================
Omnibus:                      269.886   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10799.031
Skew:                          -1.876   Prob(JB):                         0.00
Kurtosis:                      26.491   Cond. No.                         192.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b6'></a>
### Costs 2016 = Premiums 2016 + Transfers 2016

In [21]:
# INDIVIDUAL GROUP Costs 2016 = Premiums 2016 + Transfers 2016

no_nulls = ind_data[['PremiumsPMM', 'TransfersPMM', 'CostsPMM']].dropna()
X = no_nulls[['PremiumsPMM', 'TransfersPMM']]
y = no_nulls['CostsPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               CostsPMM   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     792.1
Date:                Thu, 15 Nov 2018   Prob (F-statistic):          9.53e-143
Time:                        13:49:19   Log-Likelihood:                -2551.1
No. Observations:                 420   AIC:                             5108.
Df Residuals:                     417   BIC:                             5120.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           23.4054     27.238      0.859      0.391     -30.135      76.946
PremiumsPMM      1.0120      0.072     14.117      0.000       0.871       1.153
TransfersPMM     1.3490      0.049     27.779      0.000       1.254       1.444
==============================================================================
Omnibus:                      352.898   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32911.886
Skew:                           2.921   Prob(JB):                         0.00
Kurtosis:                      45.972   Cond. No.                     2.03e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
# SMALL GROUP Costs 2016 = Premiums 2016 + Transfers 2016

no_nulls = sg_data[['PremiumsPMM', 'TransfersPMM', 'CostsPMM']].dropna()
X = no_nulls[['PremiumsPMM', 'TransfersPMM']]
y = no_nulls['CostsPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               CostsPMM   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     138.5
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           4.68e-48
Time:                        13:49:19   Log-Likelihood:                -2666.6
No. Observations:                 475   AIC:                             5339.
Df Residuals:                     472   BIC:                             5352.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           54.4131     19.452      2.797      0.005      16.190      92.636
PremiumsPMM      0.6818      0.045     15.161      0.000       0.593       0.770
TransfersPMM     0.1995      0.067      2.979      0.003       0.068       0.331
==============================================================================
Omnibus:                      450.922   Durbin-Watson:                   1.675
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31653.800
Skew:                           3.805   Prob(JB):                         0.00
Kurtosis:                      42.261   Cond. No.                     2.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<a id='b7'></a>
### Costs 2015 = Premiums 2015 + Transfers 2015

In [23]:
# INDIVIDUAL GROUP Costs 2015 = Premiums 2015 + Transfers 2015

no_nulls = ind_data[['PremiumsPMM_15', 'TransfersPMM_15', 'CostsPMM_15']].dropna()
X = no_nulls[['PremiumsPMM_15', 'TransfersPMM_15']]
y = no_nulls['CostsPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_15   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     434.6
Date:                Thu, 15 Nov 2018   Prob (F-statistic):          7.91e-106
Time:                        13:49:19   Log-Likelihood:                -2771.1
No. Observations:                 453   AIC:                             5548.
Df Residuals:                     450   BIC:                             5561.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              41.3172     25.373      1.628      0.104      -8.547      91.181
PremiumsPMM_15      0.9894      0.073     13.640      0.000       0.847       1.132
TransfersPMM_15     1.2136      0.066     18.519      0.000       1.085       1.342
==============================================================================
Omnibus:                      232.820   Durbin-Watson:                   1.818
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3636.206
Skew:                           1.822   Prob(JB):                         0.00
Kurtosis:                      16.393   Cond. No.                     1.74e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
# SMALL GROUP Costs 2015 = Premiums 2015 + Transfers 2015

no_nulls = sg_data[['PremiumsPMM_15', 'TransfersPMM_15', 'CostsPMM_15']].dropna()
X = no_nulls[['PremiumsPMM_15', 'TransfersPMM_15']]
y = no_nulls['CostsPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_15   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     203.4
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.56e-66
Time:                        13:49:19   Log-Likelihood:                -3011.9
No. Observations:                 542   AIC:                             6030.
Df Residuals:                     539   BIC:                             6043.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              31.2007     15.457      2.018      0.044       0.836      61.565
PremiumsPMM_15      0.7186      0.037     19.258      0.000       0.645       0.792
TransfersPMM_15     0.2252      0.060      3.757      0.000       0.107       0.343
==============================================================================
Omnibus:                      249.117   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2478.028
Skew:                           1.756   Prob(JB):                         0.00
Kurtosis:                      12.869   Cond. No.                     2.36e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<a id='b8'></a>
### Costs 2014 = Premiums 2014 + Transfers 2014

In [25]:
# INDIVIDUAL GROUP Costs 2014 = Premiums 2014 + Transfers 2014

no_nulls = ind_data[['PremiumsPMM_14', 'TransfersPMM_14', 'CostsPMM_14']].dropna()
X = no_nulls[['PremiumsPMM_14', 'TransfersPMM_14']]
y = no_nulls['CostsPMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_14   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     375.8
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           1.42e-92
Time:                        13:49:19   Log-Likelihood:                -2458.2
No. Observations:                 403   AIC:                             4922.
Df Residuals:                     400   BIC:                             4934.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              45.2620     19.717      2.296      0.022       6.500      84.025
PremiumsPMM_14      0.9510      0.061     15.487      0.000       0.830       1.072
TransfersPMM_14     1.4109      0.082     17.164      0.000       1.249       1.572
==============================================================================
Omnibus:                      260.126   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4245.160
Skew:                           2.452   Prob(JB):                         0.00
Kurtosis:                      18.125   Cond. No.                     1.20e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
# SMALL GROUP Costs 2014 = Premiums 2014 + Transfers 2014

no_nulls = sg_data[['PremiumsPMM_14', 'TransfersPMM_14', 'CostsPMM_14']].dropna()
X = no_nulls[['PremiumsPMM_14', 'TransfersPMM_14']]
y = no_nulls['CostsPMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_14   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     416.4
Date:                Thu, 15 Nov 2018   Prob (F-statistic):          7.65e-110
Time:                        13:49:20   Log-Likelihood:                -2858.3
No. Observations:                 539   AIC:                             5723.
Df Residuals:                     536   BIC:                             5736.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              24.5064     10.206      2.401      0.017       4.458      44.555
PremiumsPMM_14      0.7301      0.026     28.301      0.000       0.679       0.781
TransfersPMM_14     0.3895      0.073      5.361      0.000       0.247       0.532
==============================================================================
Omnibus:                       96.586   Durbin-Watson:                   1.550
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              699.507
Skew:                           0.554   Prob(JB):                    1.27e-152
Kurtosis:                       8.470   Cond. No.                     1.92e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.92e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<a id='c1'></a>
## Effect of losses in 2014 and 2015 on exiting the market in 2016

In [27]:
# Looking at the effect of losses in 2014 and 2015 on exiting the market in 2016

ind_data_exit = ind_data_14.merge(ind_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(ind_data_16, on='HIOS ID', how='left')
ind_data_exit["LossQuintile"] = pd.qcut(ind_data_exit['LossPMM_14'], 5, labels=False)
ind_data_exit["ExitedMarketIn2016"] = np.where(ind_data_exit[['TransfersPMM']].isnull(), 1, 0)
ind_data_exit = pd.concat([ind_data_exit, pd.get_dummies(ind_data_exit['LossQuintile'])], axis=1)
ind_data_exit.columns = ind_data_exit.columns.astype(str)

In [28]:
# ExitedMarketIn 2016 = Loss 2014 Q1 + Loss 2014 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = ind_data_exit[['0', '4', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['0', '4']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     ExitedMarketIn2016   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     2.918
Date:                Thu, 15 Nov 2018   Prob (F-statistic):             0.0552
Time:                        13:49:20   Log-Likelihood:                -243.04
No. Observations:                 403   AIC:                             492.1
Df Residuals:                     400   BIC:                             504.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2863      0.029     10.012      0.000       0.230       0.343
0              0.0470      0.057      0.825      0.410      -0.065       0.159
4             -0.1135      0.057     -1.990      0.047      -0.226      -0.001
==============================================================================
Omnibus:                      109.831   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               81.276
Skew:                           0.993   Prob(JB):                     2.24e-18
Kurtosis:                       2.054   Cond. No.                         3.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
# ExitedMarketIn 2016 = Loss 2014 + Loss 2015

no_nulls = ind_data_exit[['LossPMM_14', 'LossPMM_15', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['LossPMM_14', 'LossPMM_15']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     ExitedMarketIn2016   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     6.432
Date:                Thu, 15 Nov 2018   Prob (F-statistic):            0.00179
Time:                        13:49:20   Log-Likelihood:                -200.98
No. Observations:                 376   AIC:                             408.0
Df Residuals:                     373   BIC:                             419.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2520      0.022     11.216      0.000       0.208       0.296
LossPMM_14    -0.0010      0.000     -3.142      0.002      -0.002      -0.000
LossPMM_15  3.938e-05      0.000      0.147      0.883      -0.000       0.001
==============================================================================
Omnibus:                       66.184   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               99.602
Skew:                           1.254   Prob(JB):                     2.35e-22
Kurtosis:                       2.742   Cond. No.                         113.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# Looking at the effect of losses in 2014 and 2015 on exiting the market in 2016

sg_data_exit = sg_data_14.merge(sg_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(sg_data_16, on='HIOS ID', how='left')
sg_data_exit["LossQuintile"] = pd.qcut(sg_data_exit['LossPMM_14'], 5, labels=False)
sg_data_exit["ExitedMarketIn2016"] = np.where(sg_data_exit[['TransfersPMM']].isnull(), 1, 0)
sg_data_exit = pd.concat([sg_data_exit, pd.get_dummies(sg_data_exit['LossQuintile'])], axis=1)
sg_data_exit.columns = sg_data_exit.columns.astype(str)

In [31]:
# ExitedMarketIn 2016 = Loss 2014 Q1 + Loss 2014 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = sg_data_exit[['0', '4', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['0', '4']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     ExitedMarketIn2016   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     17.57
Date:                Thu, 15 Nov 2018   Prob (F-statistic):           4.06e-08
Time:                        13:49:20   Log-Likelihood:                -252.30
No. Observations:                 539   AIC:                             510.6
Df Residuals:                     536   BIC:                             523.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1238      0.022      5.744      0.000       0.081       0.166
0              0.2465      0.043      5.724      0.000       0.162       0.331
4              0.1262      0.043      2.929      0.004       0.042       0.211
==============================================================================
Omnibus:                      107.446   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              174.409
Skew:                           1.383   Prob(JB):                     1.34e-38
Kurtosis:                       3.334   Cond. No.                         3.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
# ExitedMarketIn 2016 = Loss 2014 + Loss 2015

no_nulls = sg_data_exit[['LossPMM_14', 'LossPMM_15', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['LossPMM_14', 'LossPMM_15']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     ExitedMarketIn2016   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     4.720
Date:                Thu, 15 Nov 2018   Prob (F-statistic):            0.00932
Time:                        13:49:20   Log-Likelihood:                -169.52
No. Observations:                 499   AIC:                             345.0
Df Residuals:                     496   BIC:                             357.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2042      0.027      7.547      0.000       0.151       0.257
LossPMM_14    -0.0006      0.000     -1.623      0.105      -0.001       0.000
LossPMM_15    -0.0003      0.000     -1.234      0.218      -0.001       0.000
==============================================================================
Omnibus:                      195.107   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              489.904
Skew:                           2.084   Prob(JB):                    4.16e-107
Kurtosis:                       5.489   Cond. No.                         239.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""